<a href="https://colab.research.google.com/github/jhackisneros/VISION_ARTIFICIAL/blob/main/04_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VISION_ARTIFICIAL - Notebook 04: Entrenamiento del modelo CNN

🔹 Importar librerías

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
import json, os

🔹 Cargar datos y modelo

In [ ]:
processed_dir = "../data/processed/"
x_train = np.load(os.path.join(processed_dir, "x_train.npy"))
y_train = np.load(os.path.join(processed_dir, "y_train.npy"))
x_test  = np.load(os.path.join(processed_dir, "x_test.npy"))
y_test  = np.load(os.path.join(processed_dir, "y_test.npy"))

from tensorflow.keras.models import load_model
model_path = "../models/cnn_best.keras"

try:
    model = load_model(model_path)
    print("Modelo cargado desde:", model_path)
except:
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
    print("No se encontró modelo previo, creando uno nuevo...")
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(32,32,3)),
        MaxPooling2D((2,2)),
        Conv2D(64, (3,3), activation='relu', padding='same'),
        MaxPooling2D((2,2)),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


FileNotFoundError: [Errno 2] No such file or directory: '../data/processed/x_train.npy'

🔹 Callbacks

In [ ]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint(filepath="../models/cnn_best.keras", monitor='val_accuracy', save_best_only=True)
]


🔹 Entrenamiento

In [ ]:
history = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=30,
    batch_size=64,
    callbacks=callbacks
)

🔹 Guardar historial de entrenamiento

In [ ]:
os.makedirs("../experiments", exist_ok=True)
history_dict = history.history
with open("../experiments/history_cnn.json", "w") as f:
    json.dump(history_dict, f)


🔹 Gráficas de entrenamiento

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(history.history['accuracy'], label='Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Validación')
plt.title("Precisión del modelo")
plt.xlabel("Épocas")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)
plt.savefig("../reports/figures/loss_curves.png", bbox_inches='tight')
plt.show()

print("\n✅ Entrenamiento completado. Modelo guardado en '../models/cnn_best.keras'")